# Feature Reduction and Data Preprocessing Pipeline

This notebook provides a comprehensive workflow for preparing data for machine learning and feature reduction. Follow the cells from top to bottom for a complete data preprocessing pipeline.

## 📋 Workflow Overview

1. **Setup Environment** → Install packages and import libraries
2. **Configure Settings** → Set preprocessing parameters and thresholds
3. **Load Data** → Select and upload your CSV file
4. **Initial Quality Check** → Remove columns with excessive missing data (>50% by default)
5. **Strategic Imputation** → Intelligent missing data imputation using threshold-based methods
6. **Target Selection** → Choose your dependent variable for modeling
7. **Complete Workflow** → (Optional) Run all preprocessing steps automatically

**💡 Tip**: Execute each cell in order. The pipeline builds upon previous steps and maintains data integrity throughout the process.

## Install Necessary Packages

In [34]:
# Upgrade pip to the latest version
!python.exe -m pip install --proxy http://proxy-dmz.intel.com:912  --upgrade pip

# Install necessary packages
!pip install --proxy http://proxy-dmz.intel.com:912 matplotlib pandas scikit-learn seaborn xgboost lightgbm shap catboost numpy scipy ipywidgets typing

## Load Packages

In [35]:
import pandas as pd
import numpy as np
import pickle
import joblib
import seaborn as sns
from datetime import datetime

# Import regressors
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, StackingRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, r2_score

from scipy.stats import norm, uniform, randint

import matplotlib.pyplot as plt

import tkinter as tk
from tkinter import filedialog
import os
from typing import Dict, List, Any, Optional

width = 10
height = 10
figSize = [width,height]

%matplotlib inline

## Constants

Configure all preprocessing behaviors by modifying these constants.

In [36]:
# =============================================================================
# DATA PREPROCESSING CONFIGURATION CONSTANTS
# =============================================================================

# Missing Data Thresholds
MAX_MISSING_DATA = 0.5           # Remove columns with > 50% missing data
LOW_MISSING_THRESHOLD = 0.05     # < 5% missing - Use simple imputation (median/mode)
MEDIUM_MISSING_THRESHOLD = 0.20  # < 20% missing - Use advanced imputation (KNN/iterative)
HIGH_MISSING_THRESHOLD = 0.40    # < 40% missing - Use advanced methods or consider dropping

# Data Quality Settings
REMOVE_DATE_COLUMNS = True       # Automatically remove datetime columns
HIGH_CARDINALITY_THRESHOLD = 0.8 # Remove columns with >= this unique values (likely IDs)
REMOVE_DUPLICATE_ROWS = True     # Automatically remove duplicate rows
LOW_VARIANCE_THRESHOLD = 0.99    # Remove columns with >= this value of same values
REMOVE_LOW_VARIANCE_COLS = True  # Enable automatic low variance column removal

# String Processing  
CONVERT_STRING_NULLS = True      # Convert string representations of null values to pandas NaN
STRING_NULL_VALUES = [           # Common string representations of null/missing values (case-sensitive)
    # Common NaN representations
    'NaN', 'nan', 'NAN',
    # Python None representations  
    'None', 'none', 'NONE',
    # SQL NULL representations
    'NULL', 'null', 'Null',
    # Not Available representations
    'n/a', 'N/A', 'na', 'NA',
    # Missing value indicators
    'missing', 'Missing', 'MISSING',
    # Empty and whitespace strings
    '', ' ', '  ',
    # Dash indicators
    '-', '--', '---',
    # Unknown indicators
    '?', '??', 'unknown', 'Unknown', 'UNKNOWN',
    # Additional comprehensive patterns
    'undefined', 'empty', 'blank', 'void', 'nil', 'not available',
    'not applicable', 'not specified', 'unspecified', 'not provided',
    'no data', 'no value', 'not recorded', 'not given', 'absent',
    'not found', 'not set', 'not entered', 'not supplied'
]

# Global Variables for Analysis Results
base_data = None                 # Will store the main dataset DataFrame for analysis and processing
column_info = {}                 # Will store comprehensive data analysis results from exploration

## Select and Upload Input File

In [37]:
"""
CSV File Loader Module

This module provides functions for loading CSV files with a graphical file picker
and robust error handling for common CSV file issues.

Dependencies:
    - pandas: For DataFrame operations
    - tkinter: For file dialog GUI
    - os: For file path operations
    - typing: For type hints

Example:
    Run directly in Notebook
        Ctrl-Enter
"""

def select_csv_file() -> Optional[pd.DataFrame]:
    """
    Open a file dialog to select and load a CSV file into a pandas DataFrame.
    
    This function creates a GUI file picker that allows users to select CSV files
    from their filesystem. It includes robust error handling for common issues
    like encoding problems and provides detailed information about the loaded data.
    
    The function attempts to load the CSV with UTF-8 encoding first, and falls back
    to latin-1 encoding if UTF-8 fails. It also provides comprehensive information
    about the loaded DataFrame including shape, columns, memory usage, and data types.
    
    Returns:
        Optional[pd.DataFrame]: 
            - pandas DataFrame containing the CSV data if successfully loaded
            - None if no file was selected or if an error occurred during loading
    
    Raises:
        No exceptions are raised directly. All exceptions are caught and handled
        internally, with error messages printed to console.
    
    Example:
        >>> df = select_csv_file()
        Selected file: /path/to/data.csv
        ✅ File loaded successfully!
        📊 Shape: 1,000 rows × 15 columns
        ...
        
        >>> if df is not None:
        ...     print(df.head())
    
    Note:
        - Requires tkinter for GUI (usually included with Python)
        - The file dialog starts in the user's home directory
        - Supports both .csv files and all file types in the picker
        - Uses low_memory=False to prevent mixed data type warnings
    """
    # Initialize tkinter root window for file dialog
    root = tk.Tk()
    root.withdraw()  # Hide the main tkinter window to show only dialog
    root.attributes('-topmost', True)  # Bring dialog to front of all windows
    
    try:
        # Open file selection dialog
        file_path = filedialog.askopenfilename(
            title="Select CSV File",
            filetypes=[
                ("CSV files", "*.csv"),    # Primary file type filter
                ("All files", "*.*")       # Fallback to show all files
            ],
            initialdir=os.path.expanduser("~")  # Start in user's home directory
        )
        
        # Check if user cancelled the dialog or didn't select a file
        if not file_path:
            print("No file selected.")
            return None
            
        print(f"Selected file: {file_path}")
        
        # Attempt to load CSV with error handling for encoding issues
        try:
            # Primary attempt: Load with UTF-8 encoding
            df = pd.read_csv(
                file_path, 
                low_memory=False,  # Prevents mixed data types warning
                encoding='utf-8'   # Explicit UTF-8 encoding
            )
        except UnicodeDecodeError:
            # Fallback: Try latin-1 encoding for files with special characters
            print("UTF-8 encoding failed, trying latin-1...")
            try:
                df = pd.read_csv(
                    file_path, 
                    low_memory=False, 
                    encoding='latin-1'
                )
            except Exception as e:
                print(f"❌ Encoding error: {e}")
                return None
                
        # Display success message and basic file information
        print("✅ File loaded successfully!")
        print(f"📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
        
        # Display column information (limit to first 10 for readability)
        columns = list(df.columns)
        if len(columns) <= 10:
            print(f"📋 Columns: {columns}")
        else:
            print(f"📋 First 10 columns: {columns[:10]}")
            print(f"... and {len(columns) - 10} more columns")
            
        # Calculate and display memory usage
        memory_mb = df.memory_usage(deep=True).sum() / 1024**2
        print(f"📈 Memory usage: {memory_mb:.1f} MB")
        
        # Show summary of data types in the DataFrame
        dtype_counts = df.dtypes.value_counts()
        print(f"📝 Data types: {dict(dtype_counts)}")
        
        return df
        
    except Exception as e:
        # Catch any unexpected errors during the file loading process
        print(f"❌ Unexpected error: {e}")
        return None
    finally:
        # Ensure tkinter window is properly destroyed regardless of success/failure
        root.destroy()


if __name__ == "__main__":
    """
    Main execution block - runs when script is executed directly.
    
    This automatically opens the file selection dialog and attempts to load
    a CSV file. If successful, it displays the first few rows of the data.
    """
    print("🗂️ CSV Loader Starting...")
    
    # Automatically run the file selector when script is executed
    base_data = select_csv_file()
    
    # Check if data was successfully loaded
    if base_data is not None:
        print(f"\n✅ Data loaded into 'base_data' variable")
        print("📋 First few rows:")
        print(base_data.head())
    else:
        print("\n❌ No data loaded")
        print("💡 Tip: Make sure you selected a valid CSV file")

🗂️ CSV Loader Starting...
Selected file: C:/Users/cdbrown/OneDrive - Intel Corporation/Documents/P/Python/regression_modeling/GMZ_Resistance_Data.csv
Selected file: C:/Users/cdbrown/OneDrive - Intel Corporation/Documents/P/Python/regression_modeling/GMZ_Resistance_Data.csv
✅ File loaded successfully!
📊 Shape: 12,383 rows × 212 columns
📋 First 10 columns: ['LOT', 'WAFER', 'WAFER_ID', 'FDC_SUMMARY@Max@CurrentStepTime@Process_Step05_ILDOX_EP@P1227_GTOct_CE_GM0_P1@ETCHER@255053', 'FDC_SUMMARY@Max@PinSect2TorqueDeltaBase@Process_Pin_Torque@P1227_GTOct_CE_GM0_P1@ETCHER@255053', 'FDC_SUMMARY@Max@PinSect2TorqueDeltaInteg@Process_Pin_Torque@P1227_GTOct_CE_GM0_P1@ETCHER@255053', 'FDC_SUMMARY@Max@PinSect2TorqueDeltaMax@Process_Pin_Torque@P1227_GTOct_CE_GM0_P1@ETCHER@255053', 'FDC_SUMMARY@Max@PinSect2TorqueMax@Process_Pin_Torque@P1227_GTOct_CE_GM0_P1@ETCHER@255053', 'FDC_SUMMARY@Max@RFRefPwrLow@Process_Step05_ILDOX@P1227_GTOct_CE_GM0_P1@ETCHER@255053', 'FDC_SUMMARY@Max@RFRefPwrUp@Process_Step05_IL

## Analyze Columns for Missing Data and Remove Those missing more than MAX_MISSING_DATA

In [38]:
"""
Missing Data Quality Control Module

This module analyzes and automatically removes columns from a pandas DataFrame that exceed
a specified threshold of missing data. It's designed to clean datasets before feature
engineering and machine learning model development.

Dependencies:
    - pandas (for DataFrame operations)
    - MAX_MISSING_DATA: Global variable defining the threshold (default: 0.5 = 50%)
    - base_data: Global pandas DataFrame containing the dataset to analyze

Usage:
    Ensure MAX_MISSING_DATA and base_data are defined before running this code block.
    The code will automatically remove columns exceeding the missing data threshold.

Example:
    MAX_MISSING_DATA = 0.5  # Remove columns with 50%+ missing data
    base_data = pd.read_csv('your_dataset.csv')
    # Run this code block to clean the dataset
"""

# ==============================================================================
# MISSING DATA ANALYSIS AND CLEANUP
# ==============================================================================

# Check for columns with MAX_MISSING_DATA or more missing data and remove them
if 'base_data' in locals():
    """
    Safety check to ensure the base_data DataFrame exists in the local scope
    before attempting to analyze it. Prevents NameError exceptions.
    """
    
    # -------------------------------------------------------------------------
    # STEP 1: Calculate missing data statistics
    # -------------------------------------------------------------------------
    
    # Count missing values (NaN, None, null) for each column
    missing_data = base_data.isnull().sum()
    
    # Get total number of rows for percentage calculations
    total_rows = len(base_data)
    
    # Store original column count for reporting
    original_columns = len(base_data.columns)
    
    # -------------------------------------------------------------------------
    # STEP 2: Calculate missing data percentages
    # -------------------------------------------------------------------------
    
    # Convert raw missing counts to percentages for each column
    # Formula: (missing_count / total_rows) * 100
    missing_percentages = (missing_data / total_rows) * 100
    
    # Convert the threshold from decimal (0.5) to percentage (50.0) for display
    max_missing_threshold = MAX_MISSING_DATA * 100
    
    # -------------------------------------------------------------------------
    # STEP 3: Identify columns exceeding the threshold
    # -------------------------------------------------------------------------
    
    # Find columns with missing data >= threshold
    # Uses >= to include columns with exactly the threshold amount of missing data
    high_missing_cols = missing_percentages[missing_percentages >= max_missing_threshold]
    
    # -------------------------------------------------------------------------
    # STEP 4: Display analysis results
    # -------------------------------------------------------------------------
    
    print(f"📊 Missing Data Analysis:")
    print(f"Original columns: {original_columns}")
    print(f"Columns with >={max_missing_threshold}% missing data: {len(high_missing_cols)}")
    
    # -------------------------------------------------------------------------
    # STEP 5: Process columns for removal (if any found)
    # -------------------------------------------------------------------------
    
    if len(high_missing_cols) > 0:
        """
        If columns exceed the missing data threshold, remove them from the dataset
        """
        
        # Display which columns will be removed, sorted by missing percentage (highest first)
        print(f"\n🗑️ Columns being removed:")
        for col, missing_pct in high_missing_cols.sort_values(ascending=False).items():
            print(f"  {col}: {missing_pct:.1f}% missing")
        
        # Extract column names from the pandas Series index
        columns_to_drop = high_missing_cols.index.tolist()
        
        # Remove the problematic columns from the DataFrame
        # Note: This modifies the global base_data variable
        base_data = base_data.drop(columns=columns_to_drop)
        
        # Report the results of the cleanup operation
        print(f"\n✅ Removed {len(columns_to_drop)} columns")
        print(f"Remaining columns: {len(base_data.columns)}")
        print(f"Data shape after cleanup: {base_data.shape}")
        
    else:
        """
        If no columns exceed the threshold, report successful data quality
        """
        print(f"\n✅ No columns have {max_missing_threshold}% or more missing data!")
        print(f"Data shape: {base_data.shape}")
        
else:
    """
    Error handling: Inform user that the required DataFrame doesn't exist
    """
    print("❌ Please load your data first using: base_data = select_csv_file()")

📊 Missing Data Analysis:
Original columns: 212
Columns with >=50.0% missing data: 2

🗑️ Columns being removed:
  RECIPE@NTSC@Robot-10@CVD@T_GMZILD: 100.0% missing
  END_TIME@ENTITY@NTSC@SORTER@255053: 99.6% missing

✅ Removed 2 columns
Remaining columns: 210
Data shape after cleanup: (12383, 210)


## Characterize Data

In [39]:
"""
Comprehensive Data Exploration Script
=====================================

This script performs a thorough exploration and analysis of a pandas DataFrame
to understand data structure, quality, and potential issues before analysis.

Requirements:
    - pandas library
    - A DataFrame named 'base_data' in the local namespace

"""


def comprehensive_data_exploration() -> None:
    """
    Performs comprehensive data exploration on the 'base_data' DataFrame.
    
    This function analyzes data structure, types, missing values, duplicates,
    and potential quality issues. Results are stored in a global 'column_info'
    dictionary for subsequent analysis.
    
    Features:
        - Dataset overview (shape, memory usage, duplicates)
        - Data type analysis and categorization
        - Missing value detection and quantification
        - Numeric statistics for sample columns
        - Object column uniqueness analysis
        - Mixed data type detection
        - Data quality issue identification
    
    Global Variables Created:
        column_info (dict): Comprehensive metadata about the dataset
    
    Returns:
        None: Prints analysis results and creates global column_info variable
        
    Raises:
        NameError: If 'base_data' DataFrame is not found in local namespace
        AttributeError: If 'base_data' is not a valid pandas DataFrame
    """
    
    global base_data, column_info
    
    # Validate DataFrame existence and type
    if 'base_data' not in globals():
        print("❌ base_data DataFrame not found. Please load your data first.")
        return
    
    if not hasattr(base_data, 'shape'):
        print("❌ base_data is not a valid pandas DataFrame. Please load your data first.")
        return
    
    print("=" * 80)
    print("📊 COMPREHENSIVE DATA EXPLORATION")
    print("=" * 80)
    
    # ========================================================================
    # SECTION 0: STRING NULL VALUE PREPROCESSING
    # ========================================================================
    
    # Check if string null conversion is enabled
    if 'CONVERT_STRING_NULLS' in globals() and CONVERT_STRING_NULLS:
        print(f"\n🔧 Preprocessing: Converting string null values to pandas null...")
        
        # Use centralized string null values from constants
        if 'STRING_NULL_VALUES' not in globals():
            print("❌ STRING_NULL_VALUES constant not found. Please run the constants section first.")
            return
        
        print(f"✅ Using {len(STRING_NULL_VALUES)} configured string null patterns")
        
        # Count conversions for reporting
        total_conversions = 0
        conversion_summary = {}
        
        # Process only object (string) columns to avoid converting actual numeric values
        object_columns = base_data.select_dtypes(include=['object']).columns
        
        for col in object_columns:
            # Count how many string nulls exist before conversion
            col_conversions = 0
            for null_val in STRING_NULL_VALUES:
                if null_val in base_data[col].values:
                    count_before = (base_data[col] == null_val).sum()
                    if count_before > 0:
                        col_conversions += count_before
            
            if col_conversions > 0:
                # Replace string null values with actual pandas NaN
                base_data[col] = base_data[col].replace(STRING_NULL_VALUES, np.nan)
                conversion_summary[col] = col_conversions
                total_conversions += col_conversions
        
        # Report conversion results
        if total_conversions > 0:
            print(f"✅ Converted {total_conversions:,} string null values to pandas NaN in {len(conversion_summary)} columns")
            if len(conversion_summary) <= 10:  # Show details for reasonable number of columns
                for col, count in conversion_summary.items():
                    print(f"   {col}: {count:,} conversions")
            else:
                print(f"   Top 5 columns with most conversions:")
                sorted_conversions = sorted(conversion_summary.items(), key=lambda x: x[1], reverse=True)
                for col, count in sorted_conversions[:5]:
                    print(f"   {col}: {count:,} conversions")
                print(f"   ... and {len(conversion_summary) - 5} more columns")
        else:
            print("✅ No string null values found to convert")
    else:
        print(f"\n🔧 String null conversion disabled (CONVERT_STRING_NULLS = {CONVERT_STRING_NULLS if 'CONVERT_STRING_NULLS' in globals() else 'not set'})")
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 1: DATASET OVERVIEW
    # ========================================================================
    print(f"\n📊 Dataset Overview:")
    print(f"Shape: {base_data.shape[0]:,} rows × {base_data.shape[1]:,} columns")
    print(f"Memory usage: {base_data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Check for duplicate rows (reporting only)
    duplicate_count = base_data.duplicated().sum()
    if duplicate_count > 0:
        print(f"⚠️  Duplicate rows found: {duplicate_count:,} ({duplicate_count/len(base_data)*100:.1f}%)")
        print("   (Will be addressed in data cleaning phase)")
    else:
        print("✅ No duplicate rows found")
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 2: DATA TYPES ANALYSIS
    # ========================================================================
    print("🔍 Data Types Analysis:")
    dtype_counts = base_data.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"  {dtype}: {count} columns")
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 3: MISSING VALUES ANALYSIS
    # ========================================================================
    print("❗ Missing Values Analysis:")
    missing_data = base_data.isnull().sum()
    missing_data = missing_data[missing_data > 0].sort_values(ascending=False)
    
    if len(missing_data) > 0:
        print(f"Columns with missing values: {len(missing_data)}")
        print("\nTop 20 columns with most missing values:")
        for col, missing_count in missing_data.head(20).items():
            missing_pct = (missing_count / len(base_data)) * 100
            print(f"  {col}: {missing_count:,} ({missing_pct:.1f}%)")
    else:
        print("✅ No missing values found!")
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 4: COLUMN CATEGORIZATION BY DATA TYPE
    # ========================================================================
    
    # Categorize columns by pandas data types
    float_cols = base_data.select_dtypes(include=['float64', 'float32']).columns.tolist()
    int_cols = base_data.select_dtypes(include=['int64', 'int32', 'int16', 'int8']).columns.tolist()
    object_cols = base_data.select_dtypes(include=['object']).columns.tolist()
    datetime_cols = base_data.select_dtypes(include=['datetime64']).columns.tolist()
    bool_cols = base_data.select_dtypes(include=['bool']).columns.tolist()
    
    print("📋 Columns by Data Type:")
    
    # Display float columns
    print(f"\n🔢 Float columns ({len(float_cols)}):")
    if float_cols:
        for i, col in enumerate(float_cols):
            if i % 3 == 0:
                print()
            print(f"  {col:<30}", end="")
        print()
    
    # Display integer columns
    print(f"\n🔢 Integer columns ({len(int_cols)}):")
    if int_cols:
        for i, col in enumerate(int_cols):
            if i % 3 == 0:
                print()
            print(f"  {col:<30}", end="")
        print()
    
    # Display object/string columns
    print(f"\n📝 Object/String columns ({len(object_cols)}):")
    if object_cols:
        for i, col in enumerate(object_cols):
            if i % 3 == 0:
                print()
            print(f"  {col:<30}", end="")
        print()
    
    # Display datetime columns (if any)
    if datetime_cols:
        print(f"\n📅 DateTime columns ({len(datetime_cols)}):")
        for i, col in enumerate(datetime_cols):
            if i % 3 == 0:
                print()
            print(f"  {col:<30}", end="")
        print()
    
    # Display boolean columns (if any)
    if bool_cols:
        print(f"\n✅ Boolean columns ({len(bool_cols)}):")
        for i, col in enumerate(bool_cols):
            if i % 3 == 0:
                print()
            print(f"  {col:<30}", end="")
        print()
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 5: NUMERIC STATISTICS
    # ========================================================================
    numeric_cols = float_cols + int_cols
    if numeric_cols:
        print("📈 Quick Numeric Statistics:")
        print(f"Found {len(numeric_cols)} numeric columns")
        
        # Show descriptive statistics for first 5 numeric columns
        sample_cols = numeric_cols[:5]
        if sample_cols:
            print(f"\nSample statistics (first {len(sample_cols)} numeric columns):")
            try:
                stats = base_data[sample_cols].describe()
                print(stats.round(2))
            except Exception as e:
                print(f"Error calculating statistics: {e}")
                
            if len(numeric_cols) > 5:
                print(f"\n... and {len(numeric_cols) - 5} more numeric columns")
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 6: OBJECT COLUMN ANALYSIS
    # ========================================================================
    if object_cols:
        print("📝 Object Column Analysis:")
        print(f"Analyzing all {len(object_cols)} object columns...")
        
        # Analyze ALL object columns for uniqueness and top values
        for col in object_cols:
            try:
                unique_count = base_data[col].nunique()
                total_count = len(base_data[col].dropna())
                
                if total_count > 0:
                    uniqueness_ratio = unique_count / total_count
                    print(f"  {col}: {unique_count:,} unique values ({uniqueness_ratio:.1%} unique)")
                    
                    # Show most common values
                    top_values = base_data[col].value_counts().head(3)
                    formatted_values = [f'{val} ({count})' for val, count in top_values.items()]
                    print(f"    Top values: {', '.join(formatted_values)}")
                    
            except Exception as e:
                print(f"  {col}: Error analyzing column - {e}")
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 7: MIXED DATA TYPE DETECTION (COMPREHENSIVE)
    # ========================================================================
    print("⚠️  Columns with Mixed/Problematic Data:")
    mixed_cols = []
    
    for col in object_cols:
        try:
            # Check ALL non-null values to thoroughly detect mixed types
            all_values = base_data[col].dropna()
            if len(all_values) > 0:
                # Count how many values can be converted to float
                numeric_count = 0
                for val in all_values:
                    try:
                        float(val)
                        numeric_count += 1
                    except (ValueError, TypeError):
                        pass
                
                # If some but not all values are numeric, flag as mixed
                if 0 < numeric_count < len(all_values):
                    mixed_cols.append(col)
                    unique_vals = base_data[col].nunique()
                    print(f"  {col}: {unique_vals:,} unique values "
                          f"(mixed numeric/text - {numeric_count:,}/{len(all_values):,} numeric)")
                          
        except Exception as e:
            print(f"  {col}: Error checking mixed types - {e}")
    
    if not mixed_cols:
        print("✅ No obviously mixed-type columns detected")
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 8: DATA QUALITY ISSUES
    # ========================================================================
    print("🔍 Potential Data Quality Issues:")
    issues_found = False
    
    # Check for high cardinality columns (potentially IDs)
    high_cardinality_cols = []
    for col in object_cols:
        try:
            unique_ratio = base_data[col].nunique() / len(base_data)
            if unique_ratio > HIGH_CARDINALITY_THRESHOLD:
                high_cardinality_cols.append((col, unique_ratio))
                issues_found = True
        except Exception as e:
            print(f"  Error analyzing {col}: {e}")
    
    if high_cardinality_cols:
        print("  High cardinality columns (may be IDs or need special handling):")
        for col, ratio in high_cardinality_cols:
            print(f"    {col}: {ratio:.1%} unique values")
    
    # Check for constant value columns (no variance)
    low_variance_cols = []
    for col in numeric_cols:
        try:
            if base_data[col].nunique() == 1:
                low_variance_cols.append(col)
                issues_found = True
        except Exception as e:
            print(f"  Error analyzing variance for {col}: {e}")
    
    if low_variance_cols:
        print("  Constant value columns (no variance):")
        for col in low_variance_cols:
            try:
                constant_value = base_data[col].iloc[0]
                print(f"    {col}: constant value = {constant_value}")
            except Exception as e:
                print(f"    {col}: Error retrieving constant value - {e}")
    
    if not issues_found:
        print("✅ No obvious data quality issues detected")
    
    print("\n" + "=" * 80 + "\n")
    
    # ========================================================================
    # SECTION 9: RESULTS COMPILATION
    # ========================================================================
    
    # Calculate total missing data percentage
    total_cells = base_data.shape[0] * base_data.shape[1]
    total_missing = base_data.isnull().sum().sum()
    total_missing_pct = (total_missing / total_cells) * 100 if total_cells > 0 else 0
    
    # Compile comprehensive metadata dictionary
    column_info = {
        'shape': base_data.shape,
        'memory_mb': base_data.memory_usage(deep=True).sum() / 1024**2,
        'duplicates': duplicate_count,
        'float_cols': float_cols,
        'int_cols': int_cols,
        'object_cols': object_cols,
        'datetime_cols': datetime_cols,
        'bool_cols': bool_cols,
        'mixed_cols': mixed_cols,
        'high_cardinality_cols': [col for col, _ in high_cardinality_cols],
        'low_variance_cols': low_variance_cols,
        'missing_data': missing_data,
        'total_missing_pct': total_missing_pct
    }
    
    # ========================================================================
    # SECTION 10: SUMMARY
    # ========================================================================
    print("🔍 Data exploration complete!")
    print(f"Summary: {base_data.shape[0]:,} rows, {base_data.shape[1]:,} cols, "
          f"{column_info['total_missing_pct']:.1f}% missing data")
    print("Results stored in 'column_info' variable for further analysis.")
    print("=" * 80)


# ============================================================================
# EXECUTION BLOCK
# ============================================================================

print("✅ Comprehensive data exploration function loaded!")

# Check if base_data exists and provide status
if 'base_data' in globals():
    if hasattr(base_data, 'shape'):
        print(f"✅ base_data found: {base_data.shape[0]:,} rows × {base_data.shape[1]:,} columns")
        print("🚀 Running comprehensive data exploration...")
        comprehensive_data_exploration()
    else:
        print("⚠️ base_data exists but is not a valid DataFrame")
        print("📊 To run analysis manually: comprehensive_data_exploration()")
else:
    print("⚠️ base_data not found. Please load your data first.")
    print("📊 To run analysis manually: comprehensive_data_exploration()")

print(f"\n🔧 Current configuration:")
if 'HIGH_CARDINALITY_THRESHOLD' in globals():
    print(f"   - HIGH_CARDINALITY_THRESHOLD = {HIGH_CARDINALITY_THRESHOLD}")
if 'REMOVE_DUPLICATE_ROWS' in globals():
    print(f"   - REMOVE_DUPLICATE_ROWS = {REMOVE_DUPLICATE_ROWS}")
if 'CONVERT_STRING_NULLS' in globals():
    print(f"   - CONVERT_STRING_NULLS = {CONVERT_STRING_NULLS}")

✅ Comprehensive data exploration function loaded!
✅ base_data found: 12,383 rows × 210 columns
🚀 Running comprehensive data exploration...
📊 COMPREHENSIVE DATA EXPLORATION

🔧 Preprocessing: Converting string null values to pandas null...
✅ Using 46 configured string null patterns
✅ No string null values found to convert



📊 Dataset Overview:
Shape: 12,383 rows × 210 columns
Memory usage: 58.47 MB
✅ No duplicate rows found


🔍 Data Types Analysis:
  float64: 121 columns
  object: 55 columns
  int64: 34 columns


❗ Missing Values Analysis:
Columns with missing values: 21

Top 20 columns with most missing values:
  R4_16MA[ECD_GMZ_1000_SL]@50%@ETEST: 76 (0.6%)
  PRODUCT@ETEST: 76 (0.6%)
  CHAMBER@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  SUBENTITY@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  RECIPE@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  PROCESS_TIME@CHAMBER@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  SLOT@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  END_TIME@CHAMBER@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%

## Intelligent Imputation Function

Define the intelligent imputation strategy function that will be used in the strategic workflow below.

In [40]:
def intelligent_imputation_strategy(df, column_info, target_var=None):
    """
    Intelligent imputation strategy based on data analysis and missing patterns.
    
    This function analyzes missing data patterns and applies appropriate imputation
    methods based on data types, missing percentages, and relationships with target variable.
    
    Parameters:
        df (pd.DataFrame): DataFrame to impute
        column_info (dict): Results from comprehensive_data_exploration
        target_var (str): Name of target variable (to avoid imputing it)
    
    Returns:
        pd.DataFrame: DataFrame with imputed values
        dict: Imputation report with methods used
    """
    
    print("🔧 INTELLIGENT IMPUTATION STRATEGY")
    print("=" * 60)
    print(f"✅ Using configured thresholds: Low={LOW_MISSING_THRESHOLD:.0%}, Medium={MEDIUM_MISSING_THRESHOLD:.0%}, High={HIGH_MISSING_THRESHOLD:.0%}")
    
    imputed_df = df.copy()
    imputation_report = {
        'original_missing': df.isnull().sum().sum(),
        'methods_used': {},
        'columns_imputed': [],
        'columns_skipped': [],
        'imputation_summary': {}
    }
    
    # Get missing data analysis
    missing_data = df.isnull().sum()
    missing_cols = missing_data[missing_data > 0].sort_values(ascending=False)
    
    if len(missing_cols) == 0:
        print("✅ No missing values found!")
        return imputed_df, imputation_report
    
    print(f"📊 Found {len(missing_cols)} columns with missing values")
    print(f"Total missing values: {missing_data.sum():,}")
    
    # Categorize columns by data type for appropriate imputation
    numeric_cols = df.select_dtypes(include=['float64', 'float32', 'int64', 'int32']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    # Remove target variable from imputation if specified
    if target_var:
        if target_var in numeric_cols:
            numeric_cols.remove(target_var)
        if target_var in categorical_cols:
            categorical_cols.remove(target_var)
        print(f"🎯 Protecting target variable '{target_var}' from imputation")
    
    # ========================================================================
    # STEP 1: Analyze Missing Data Patterns
    # ========================================================================
    print(f"\n📋 Missing Data Analysis by Column Type:")
    
    numeric_missing = [col for col in missing_cols.index if col in numeric_cols]
    categorical_missing = [col for col in missing_cols.index if col in categorical_cols]
    
    print(f"  Numeric columns with missing values: {len(numeric_missing)}")
    print(f"  Categorical columns with missing values: {len(categorical_missing)}")
    
    # ========================================================================
    # STEP 2: Imputation Strategy Selection
    # ========================================================================
    print(f"\n🎯 Imputation Strategy Selection:")
    print(f"  📊 Low Missing (<{LOW_MISSING_THRESHOLD:.0%}): Simple imputation (median/mode)")
    print(f"  🧠 Medium Missing ({LOW_MISSING_THRESHOLD:.0%}-{MEDIUM_MISSING_THRESHOLD:.0%}): Advanced imputation (KNN/iterative)")
    print(f"  ⚠️ High Missing ({MEDIUM_MISSING_THRESHOLD:.0%}-{HIGH_MISSING_THRESHOLD:.0%}): Advanced methods or consider dropping")
    print(f"  🚨 Very High Missing (>{HIGH_MISSING_THRESHOLD:.0%}): Recommend dropping")
    
    # Use global strategy thresholds from constants section
    strategies = {
        'low_missing': [],      # Simple imputation
        'medium_missing': [],   # Advanced imputation
        'high_missing': [],     # Consider dropping or advanced methods
        'very_high_missing': [] # Recommend dropping
    }
    
    for col in missing_cols.index:
        missing_pct = missing_cols[col] / len(df)
        
        if missing_pct < LOW_MISSING_THRESHOLD:
            strategies['low_missing'].append((col, missing_pct))
        elif missing_pct < MEDIUM_MISSING_THRESHOLD:
            strategies['medium_missing'].append((col, missing_pct))
        elif missing_pct < HIGH_MISSING_THRESHOLD:
            strategies['high_missing'].append((col, missing_pct))
        else:
            strategies['very_high_missing'].append((col, missing_pct))
    
    # Report strategy assignments
    for strategy, cols in strategies.items():
        if cols:
            print(f"\n  {strategy.replace('_', ' ').title()}: {len(cols)} columns")
            for col, pct in cols[:3]:  # Show first 3
                print(f"    {col}: {pct:.1%} missing")
            if len(cols) > 3:
                print(f"    ... and {len(cols)-3} more")
    
    # ========================================================================
    # STEP 3: Apply Imputation Methods
    # ========================================================================
    print(f"\n🔄 Applying Imputation Methods:")
    
    # Import required libraries
    from sklearn.impute import SimpleImputer, KNNImputer
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer
    
    # 3A: Low Missing - Simple Imputation
    if strategies['low_missing']:
        print(f"\n📊 Simple Imputation (Low Missing < {LOW_MISSING_THRESHOLD:.0%}):")
        
        for col, missing_pct in strategies['low_missing']:
            if col == target_var:  # Skip target variable
                continue
                
            if col in numeric_cols:
                # Use median for numeric columns (robust to outliers)
                median_value = imputed_df[col].median()
                imputed_df[col] = imputed_df[col].fillna(median_value)
                method = f"median ({median_value:.3f})"
                
            elif col in categorical_cols:
                # Use mode for categorical columns
                mode_value = imputed_df[col].mode()
                if len(mode_value) > 0:
                    imputed_df[col] = imputed_df[col].fillna(mode_value[0])
                    method = f"mode ('{mode_value[0]}')"
                else:
                    imputed_df[col] = imputed_df[col].fillna('Unknown')
                    method = "constant ('Unknown')"
            
            imputation_report['methods_used'][col] = method
            imputation_report['columns_imputed'].append(col)
            print(f"  ✅ {col}: {method}")
    
    # 3B: Medium Missing - Advanced Imputation
    if strategies['medium_missing']:
        print(f"\n🧠 Advanced Imputation (Medium Missing {LOW_MISSING_THRESHOLD:.0%}-{MEDIUM_MISSING_THRESHOLD:.0%}):")
        
        # Separate numeric and categorical for different methods
        medium_numeric = [col for col, _ in strategies['medium_missing'] if col in numeric_cols and col != target_var]
        medium_categorical = [col for col, _ in strategies['medium_missing'] if col in categorical_cols and col != target_var]
        
        # KNN Imputation for numeric columns
        if medium_numeric:
            print(f"  🔢 KNN Imputation for {len(medium_numeric)} numeric columns:")
            
            # Select features for KNN (use other numeric columns with low missing)
            knn_features = [col for col in numeric_cols if col not in medium_numeric and missing_data[col] < len(df) * 0.1]
            
            if len(knn_features) >= 2:  # Need at least 2 features for KNN
                try:
                    # Prepare data for KNN
                    knn_data = imputed_df[knn_features + medium_numeric].copy()
                    
                    # Use KNN imputer
                    knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
                    knn_imputed = knn_imputer.fit_transform(knn_data)
                    
                    # Update the original dataframe
                    for i, col in enumerate(medium_numeric):
                        col_idx = knn_features.index(col) if col in knn_features else len(knn_features) + medium_numeric.index(col)
                        imputed_df[col] = knn_imputed[:, col_idx]
                        imputation_report['methods_used'][col] = "KNN (k=5)"
                        imputation_report['columns_imputed'].append(col)
                        print(f"    ✅ {col}: KNN imputation")
                        
                except Exception as e:
                    print(f"    ⚠️ KNN failed, falling back to median imputation: {e}")
                    # Fallback to median
                    for col in medium_numeric:
                        median_value = imputed_df[col].median()
                        imputed_df[col] = imputed_df[col].fillna(median_value)
                        imputation_report['methods_used'][col] = f"median fallback ({median_value:.3f})"
                        imputation_report['columns_imputed'].append(col)
            else:
                print(f"    ⚠️ Insufficient features for KNN, using median")
                for col in medium_numeric:
                    median_value = imputed_df[col].median()
                    imputed_df[col] = imputed_df[col].fillna(median_value)
                    imputation_report['methods_used'][col] = f"median ({median_value:.3f})"
                    imputation_report['columns_imputed'].append(col)
        
        # Frequent value imputation for categorical
        if medium_categorical:
            print(f"  📝 Frequent Value Imputation for {len(medium_categorical)} categorical columns:")
            for col in medium_categorical:
                # Use most frequent value
                value_counts = imputed_df[col].value_counts()
                if len(value_counts) > 0:
                    most_frequent = value_counts.index[0]
                    imputed_df[col] = imputed_df[col].fillna(most_frequent)
                    imputation_report['methods_used'][col] = f"most_frequent ('{most_frequent}')"
                    imputation_report['columns_imputed'].append(col)
                    print(f"    ✅ {col}: Most frequent ('{most_frequent}')")
    
    # 3C: High Missing - Advanced Methods or User Decision
    if strategies['high_missing']:
        print(f"\n⚠️ High Missing Data ({MEDIUM_MISSING_THRESHOLD:.0%}-{HIGH_MISSING_THRESHOLD:.0%}) - Requires Decision:")
        
        for col, missing_pct in strategies['high_missing']:
            if col == target_var:
                continue
                
            print(f"  {col}: {missing_pct:.1%} missing")
            
            # For high missing, offer choice or use conservative approach
            if col in numeric_cols:
                # Use iterative imputation (MICE-like)
                try:
                    # Create subset for iterative imputation
                    iter_features = [c for c in numeric_cols if missing_data[c] < len(df) * 0.3 and c != col]
                    
                    if len(iter_features) >= 3:
                        iter_data = imputed_df[[col] + iter_features].copy()
                        iter_imputer = IterativeImputer(random_state=42, max_iter=10)
                        iter_imputed = iter_imputer.fit_transform(iter_data)
                        imputed_df[col] = iter_imputed[:, 0]
                        imputation_report['methods_used'][col] = "iterative_imputation"
                        imputation_report['columns_imputed'].append(col)
                        print(f"    ✅ Applied iterative imputation")
                    else:
                        # Fallback to median
                        median_value = imputed_df[col].median()
                        imputed_df[col] = imputed_df[col].fillna(median_value)
                        imputation_report['methods_used'][col] = f"median_fallback ({median_value:.3f})"
                        imputation_report['columns_imputed'].append(col)
                        print(f"    ⚠️ Used median fallback")
                        
                except Exception as e:
                    # Final fallback
                    median_value = imputed_df[col].median()
                    imputed_df[col] = imputed_df[col].fillna(median_value)
                    imputation_report['methods_used'][col] = f"median_error_fallback ({median_value:.3f})"
                    imputation_report['columns_imputed'].append(col)
                    print(f"    ⚠️ Error in advanced imputation, used median: {e}")
            
            elif col in categorical_cols:
                # For categorical with high missing, create "Missing" category
                imputed_df[col] = imputed_df[col].fillna('Missing_Imputed')
                imputation_report['methods_used'][col] = "missing_category"
                imputation_report['columns_imputed'].append(col)
                print(f"    ✅ Created 'Missing_Imputed' category")
    
    # 3D: Very High Missing - Recommend Dropping
    if strategies['very_high_missing']:
        print(f"\n🚨 Very High Missing Data (>{HIGH_MISSING_THRESHOLD:.0%}) - Consider Dropping:")
        for col, missing_pct in strategies['very_high_missing']:
            print(f"  {col}: {missing_pct:.1%} missing - Consider removing this column")
            imputation_report['columns_skipped'].append(col)
    
    # ========================================================================
    # STEP 4: Final Report
    # ========================================================================
    final_missing = imputed_df.isnull().sum().sum()
    imputation_report['final_missing'] = final_missing
    imputation_report['missing_reduced'] = imputation_report['original_missing'] - final_missing
    
    print(f"\n" + "=" * 60)
    print("✅ IMPUTATION COMPLETE!")
    print(f"Original missing values: {imputation_report['original_missing']:,}")
    print(f"Final missing values: {final_missing:,}")
    print(f"Missing values imputed: {imputation_report['missing_reduced']:,}")
    print(f"Columns imputed: {len(imputation_report['columns_imputed'])}")
    print(f"Columns skipped (high missing): {len(imputation_report['columns_skipped'])}")
    
    if final_missing > 0:
        remaining_missing = imputed_df.isnull().sum()
        remaining_missing = remaining_missing[remaining_missing > 0]
        print(f"\n⚠️ Remaining missing values in:")
        for col, count in remaining_missing.items():
            pct = count / len(imputed_df) * 100
            print(f"  {col}: {count} ({pct:.1f}%)")
    
    print("=" * 60)
    
    return imputed_df, imputation_report


# Usage example
def apply_imputation_workflow():
    """
    Apply imputation as part of the preprocessing workflow
    """
    if 'base_data' in globals() and 'column_info' in globals():
        print("🔧 Applying intelligent imputation strategy...")
        
        # Apply imputation before automated cleaning
        imputed_data, imputation_report = intelligent_imputation_strategy(
            base_data, column_info, target_var=None
        )
        
        print(f"\n📊 Imputation Summary:")
        print(f"  Columns imputed: {len(imputation_report['columns_imputed'])}")
        print(f"  Missing values reduced: {imputation_report['missing_reduced']:,}")
        print(f"  Methods used: {len(set(imputation_report['methods_used'].values()))}")
        
        return imputed_data, imputation_report
    else:
        print("❌ Please run comprehensive_data_exploration() first")
        return None, None

print("🔧 Imputation functions ready!")
print("💡 Usage options:")
print("1. imputed_data, report = intelligent_imputation_strategy(base_data, column_info)")
print("2. imputed_data, report = apply_imputation_workflow()")

🔧 Imputation functions ready!
💡 Usage options:
1. imputed_data, report = intelligent_imputation_strategy(base_data, column_info)
2. imputed_data, report = apply_imputation_workflow()


## 🎯 Strategic Imputation (Recommended Decision Point)

**Perfect timing!** Now that we have the `intelligent_imputation_strategy` function defined above and complete data analysis results, this is the optimal point to apply intelligent imputation before automated cleaning. This preserves valuable columns that might otherwise be dropped.

In [41]:
# RECOMMENDED WORKFLOW: Strategic Imputation at Decision Point 1
# ================================================================

def execute_recommended_imputation_workflow():
    """
    Execute the recommended imputation strategy at the optimal decision point.
    
    This runs after comprehensive_data_exploration() but before automated_data_cleaning()
    to preserve valuable columns that might otherwise be dropped due to missing data.
    """
    
    print("🚀 EXECUTING RECOMMENDED IMPUTATION WORKFLOW")
    print("=" * 70)
    
    # Step 1: Validate prerequisites
    if 'base_data' not in globals():
        print("❌ base_data not found. Please run the file selection cell first.")
        return None, None
    
    if 'column_info' not in globals():
        print("❌ column_info not found. Please run comprehensive_data_exploration() first.")
        return None, None
    
    print("✅ Prerequisites met - data and analysis results available")
    print(f"📊 Working with dataset: {base_data.shape}")
    
    # Step 2: Apply strategic imputation
    print(f"\n🎯 Step 1: Applying intelligent imputation strategy...")
    print("📋 This will preserve valuable columns before automated cleaning")
    
    try:
        # Apply the intelligent imputation strategy
        imputed_data, imputation_report = intelligent_imputation_strategy(
            base_data, 
            column_info, 
            target_var=None  # Target not selected yet
        )
        
        print(f"\n✅ Imputation completed successfully!")
        
        # Report imputation results
        print(f"\n📈 Imputation Results Summary:")
        print(f"  Original missing values: {imputation_report['original_missing']:,}")
        print(f"  Final missing values: {imputation_report['final_missing']:,}")
        print(f"  Values imputed: {imputation_report['missing_reduced']:,}")
        print(f"  Columns processed: {len(imputation_report['columns_imputed'])}")
        print(f"  Columns skipped (high missing): {len(imputation_report['columns_skipped'])}")
        
        # Show methods used
        if imputation_report['methods_used']:
            method_counts = {}
            for method in imputation_report['methods_used'].values():
                method_type = method.split('(')[0].strip()  # Extract method name
                method_counts[method_type] = method_counts.get(method_type, 0) + 1
            
            print(f"\n🔧 Imputation Methods Applied:")
            for method, count in method_counts.items():
                print(f"  {method}: {count} columns")
        
        # Data quality comparison
        print(f"\n📊 Data Quality Improvement:")
        
        # Calculate missing values instead of completeness for clearer reporting
        original_missing_count = base_data.isnull().sum().sum()
        new_missing_count = imputed_data.isnull().sum().sum()
        total_cells = base_data.shape[0] * base_data.shape[1]
        
        original_missing_pct = (original_missing_count / total_cells) * 100
        new_missing_pct = (new_missing_count / total_cells) * 100
        
        print(f"  Original missing values: {original_missing_count:,} ({original_missing_pct:.3f}% of data)")
        print(f"  After imputation: {new_missing_count:,} ({new_missing_pct:.3f}% of data)")
        print(f"  Missing values eliminated: {original_missing_count - new_missing_count:,}")
        
        if new_missing_count == 0:
            print(f"  ✅ All missing values successfully imputed!")
        
        return imputed_data, imputation_report
        
    except Exception as e:
        print(f"❌ Error during imputation: {e}")
        print("💡 You can still proceed with the original data and automated cleaning")
        return None, None

# Execute the recommended workflow
print("🎯 Ready to execute recommended imputation strategy!")
print("✅ intelligent_imputation_strategy function is now defined above and ready to use!")
print("\n📋 This is the optimal decision point because:")
print("  ✅ Complete data analysis is available (string nulls converted)")
print("  ✅ Can preserve valuable columns before automated cleaning")
print("  ✅ Advanced imputation methods can use all available information")
print("  ✅ Results will inform subsequent cleaning decisions")

print(f"\n💡 Execute the workflow:")
print("imputed_data, imputation_report = execute_recommended_imputation_workflow()")

# Uncomment the line below to run automatically:
imputed_data, imputation_report = execute_recommended_imputation_workflow()

🎯 Ready to execute recommended imputation strategy!
✅ intelligent_imputation_strategy function is now defined above and ready to use!

📋 This is the optimal decision point because:
  ✅ Complete data analysis is available (string nulls converted)
  ✅ Can preserve valuable columns before automated cleaning
  ✅ Advanced imputation methods can use all available information
  ✅ Results will inform subsequent cleaning decisions

💡 Execute the workflow:
imputed_data, imputation_report = execute_recommended_imputation_workflow()
🚀 EXECUTING RECOMMENDED IMPUTATION WORKFLOW
✅ Prerequisites met - data and analysis results available
📊 Working with dataset: (12383, 210)

🎯 Step 1: Applying intelligent imputation strategy...
📋 This will preserve valuable columns before automated cleaning
🔧 INTELLIGENT IMPUTATION STRATEGY
✅ Using configured thresholds: Low=5%, Medium=20%, High=40%
📊 Found 21 columns with missing values
Total missing values: 208

📋 Missing Data Analysis by Column Type:
  Numeric colum

## ✅ Implemented: Recommended Imputation Strategy

**Great choice!** I've implemented the recommended imputation strategy at Decision Point 1. Here's what's now available:

### 🎯 **Strategic Imputation (Decision Point 1)**
- **Location**: Right here - after comprehensive analysis, before automated cleaning
- **Function**: `execute_recommended_imputation_workflow()`
- **Benefits**: Preserves valuable columns, uses complete analysis results, informs cleaning decisions

### 🔄 **Updated Complete Workflow**  
The `complete_data_preprocessing_workflow()` function now includes:
1. **Data Loading & Validation**
2. **Comprehensive Analysis** (includes string null conversion)
3. **Strategic Imputation** ← NEW! (Decision Point 1)
4. **Automated Cleaning** (now works on imputed data)
5. **Target Selection Prep**
6. **Quality Reporting**

### 🚀 **Ready to Execute Options**

**Option 1: Strategic Imputation Only**
```python
# Execute just the recommended imputation step
imputed_data, imputation_report = execute_recommended_imputation_workflow()
```

**Option 2: Complete Enhanced Workflow**  
```python
# Execute the complete workflow with strategic imputation
cleaned_data, results = complete_data_preprocessing_workflow()
```

### 💡 **Why This Strategy Works**
- ✅ **Optimal Timing**: Has complete data analysis but before cleaning decisions
- ✅ **Data Preservation**: Saves columns that might otherwise be dropped
- ✅ **Intelligent Methods**: Uses advanced imputation based on data characteristics  
- ✅ **Informed Decisions**: Imputation results guide subsequent cleaning
- ✅ **Flexible**: Can still proceed with original workflow if needed

**Execute the cell below to proceed with your recommended strategy!**

## 📋 When to Consider Imputation: Decision Framework

### 🎯 **Key Decision Points in Your Pipeline**

**Point 1: After Comprehensive Analysis (Pre-Cleaning)**
- **Best for**: Preserving valuable columns that would otherwise be dropped
- **Use when**: Columns have 20-40% missing but contain important information
- **Methods**: KNN, Iterative imputation, domain-specific approaches

**Point 2: During Automated Cleaning (Alternative to Dropping)**  
- **Best for**: Integrated workflow with automatic decision making
- **Use when**: You want a complete end-to-end automated pipeline
- **Methods**: Intelligent strategy selection based on missing percentage

**Point 3: After Target Selection (Pre-Modeling)**
- **Best for**: Final cleanup before feature engineering
- **Use when**: Small amounts of missing data remain after cleaning
- **Methods**: Simple imputation (median, mode) for remaining gaps

### 🧠 **Imputation vs Deletion Decision Matrix**

| Missing % | Numeric Columns | Categorical Columns | Recommendation |
|-----------|----------------|-------------------|----------------|
| < 5% | Median/Mean | Mode | ✅ **Simple Imputation** |
| 5-20% | KNN/Iterative | Most Frequent | ✅ **Advanced Imputation** |
| 20-40% | MICE/Iterative | Missing Category | ⚠️ **Consider Domain Knowledge** |
| > 40% | Drop Column | Drop Column | ❌ **Usually Drop** |

### 🔧 **Method Selection Guidelines**

**Numeric Data**:
- **Mean**: Use when data is normally distributed, no outliers
- **Median**: Use when data has outliers or is skewed (recommended default)
- **KNN**: Use when similar observations can inform missing values
- **Iterative (MICE)**: Use when multiple variables have missing data patterns

**Categorical Data**:
- **Mode**: Use for ordinal data or when most frequent makes sense
- **Missing Category**: Use when "missing" itself is informative
- **Predictive**: Use when other variables can predict the category

### ⚖️ **Trade-offs to Consider**

**Advantages of Imputation**:
- ✅ Preserves sample size
- ✅ Retains potentially valuable features
- ✅ Avoids bias from complete case analysis
- ✅ Better model performance with sufficient data

**Disadvantages of Imputation**:
- ❌ Can introduce bias if done incorrectly
- ❌ May reduce variance artificially
- ❌ Computational overhead for advanced methods
- ❌ Risk of overfitting to imputation model

### 🚨 **When NOT to Impute**

1. **Missing Not at Random (MNAR)**: When missingness is informative
2. **High Missing %**: When > 50% of values are missing
3. **ID Columns**: When missing values indicate invalid records  
4. **Time Dependencies**: When imputation would violate temporal relationships
5. **Domain Constraints**: When imputed values would be impossible/invalid

## Select Target Variable for Modeling

Choose your dependent variable (the column you want to predict) from the cleaned dataset.

In [42]:
def select_dependent_variable(df):
    """
    Interactive selection of dependent variable
    """
    print("🎯 Select Dependent Variable (Target for Modeling):")
    print("\nAvailable numeric columns (likely candidates for dependent variable):")
    
    numeric_cols = df.select_dtypes(include=['float64', 'float32', 'int64', 'int32']).columns.tolist()
    
    for i, col in enumerate(numeric_cols, 1):
        non_null_count = df[col].count()
        total_count = len(df)
        missing_pct = ((total_count - non_null_count) / total_count) * 100
        mean_val = df[col].mean() if df[col].dtype in ['float64', 'float32'] else 'N/A'
        
        print(f"{i:2d}. {col}")
        print(f"    Non-null: {non_null_count}/{total_count} ({100-missing_pct:.1f}%)")
        if mean_val != 'N/A':
            print(f"    Mean: {mean_val:.3f}, Std: {df[col].std():.3f}")
        print()
    
    while True:
        try:
            choice = input(f"\nEnter the number (1-{len(numeric_cols)}) for your dependent variable: ")
            choice_idx = int(choice) - 1
            if 0 <= choice_idx < len(numeric_cols):
                selected_col = numeric_cols[choice_idx]
                print(f"✅ Selected dependent variable: {selected_col}")
                return selected_col
            else:
                print("❌ Invalid choice. Please try again.")
        except ValueError:
            print("❌ Please enter a valid number.")
        except KeyboardInterrupt:
            print("\n❌ Selection cancelled.")
            return None

# Interactive selection of dependent variable
if 'imputed_data' in locals():
    print("\n🎯 Starting dependent variable selection...")
    print("💡 Using imputed_data (data after intelligent imputation)")
    dependent_var = select_dependent_variable(imputed_data)
elif 'base_data' in locals():
    print("\n🎯 Starting dependent variable selection...")
    print("💡 Using base_data (no imputation applied yet)")
    dependent_var = select_dependent_variable(base_data)
else:
    print("❌ Please load data first (run the file selection cell).")


🎯 Starting dependent variable selection...
💡 Using imputed_data (data after intelligent imputation)
🎯 Select Dependent Variable (Target for Modeling):

Available numeric columns (likely candidates for dependent variable):
 1. WAFER
    Non-null: 12383/12383 (100.0%)

 2. FDC_SUMMARY@Max@CurrentStepTime@Process_Step05_ILDOX_EP@P1227_GTOct_CE_GM0_P1@ETCHER@255053
    Non-null: 12383/12383 (100.0%)
    Mean: 301.723, Std: 4.865

 3. FDC_SUMMARY@Max@PinSect2TorqueDeltaBase@Process_Pin_Torque@P1227_GTOct_CE_GM0_P1@ETCHER@255053
    Non-null: 12383/12383 (100.0%)
    Mean: 186.783, Std: 35.397

 4. FDC_SUMMARY@Max@PinSect2TorqueDeltaInteg@Process_Pin_Torque@P1227_GTOct_CE_GM0_P1@ETCHER@255053
    Non-null: 12383/12383 (100.0%)
    Mean: 160.661, Std: 93.413

 5. FDC_SUMMARY@Max@PinSect2TorqueDeltaMax@Process_Pin_Torque@P1227_GTOct_CE_GM0_P1@ETCHER@255053
    Non-null: 12383/12383 (100.0%)
    Mean: 21.479, Std: 9.097

 6. FDC_SUMMARY@Max@PinSect2TorqueMax@Process_Pin_Torque@P1227_GTOct_CE_G

## Complete Workflow Execution

Execute this cell to run the complete data preprocessing workflow automatically.

In [43]:
def automated_data_cleaning(df, column_info, interactive=True):
    """
    Automated data cleaning based on configuration constants and analysis results.
    
    This function performs systematic cleaning operations:
    - Removes columns with excessive missing data
    - Removes duplicate rows
    - Removes low variance columns
    - Removes datetime columns
    - Removes high cardinality columns (likely IDs)
    
    Parameters:
        df (pd.DataFrame): DataFrame to clean
        column_info (dict): Results from comprehensive_data_exploration
        interactive (bool): If True, asks for confirmation before removing columns
    
    Returns:
        tuple: (cleaned_df, cleaning_summary)
    """
    
    print("🧹 AUTOMATED DATA CLEANING")
    print("=" * 60)
    
    cleaned_df = df.copy()
    cleaning_summary = {
        'original_shape': df.shape,
        'columns_removed': [],
        'rows_removed': 0,
        'operations': []
    }
    
    # ========================================================================
    # STEP 1: Remove columns with excessive missing data
    # ========================================================================
    print(f"\n📊 Step 1: Checking for columns with >{MAX_MISSING_DATA:.0%} missing data...")
    
    missing_pct = cleaned_df.isnull().sum() / len(cleaned_df)
    high_missing_cols = missing_pct[missing_pct > MAX_MISSING_DATA].index.tolist()
    
    if high_missing_cols:
        print(f"   Found {len(high_missing_cols)} columns exceeding threshold:")
        for col in high_missing_cols[:10]:
            pct = missing_pct[col]
            print(f"      {col}: {pct:.1%} missing")
        if len(high_missing_cols) > 10:
            print(f"      ... and {len(high_missing_cols)-10} more")
        
        remove = True
        if interactive:
            response = input(f"\n   Remove these {len(high_missing_cols)} columns? (y/n): ").lower().strip()
            remove = (response == 'y')
        
        if remove:
            cleaned_df = cleaned_df.drop(columns=high_missing_cols)
            cleaning_summary['columns_removed'].extend(high_missing_cols)
            cleaning_summary['operations'].append(f"Removed {len(high_missing_cols)} columns with >{MAX_MISSING_DATA:.0%} missing data")
            print(f"   ✅ Removed {len(high_missing_cols)} high-missing columns")
        else:
            print(f"   ⏭️  Skipped removal")
    else:
        print(f"   ✅ No columns exceed {MAX_MISSING_DATA:.0%} missing data threshold")
    
    # ========================================================================
    # STEP 2: Remove duplicate rows
    # ========================================================================
    if REMOVE_DUPLICATE_ROWS:
        print(f"\n🔄 Step 2: Checking for duplicate rows...")
        
        duplicates = cleaned_df.duplicated().sum()
        if duplicates > 0:
            print(f"   Found {duplicates} duplicate rows ({duplicates/len(cleaned_df):.1%})")
            
            remove = True
            if interactive:
                response = input(f"   Remove duplicate rows? (y/n): ").lower().strip()
                remove = (response == 'y')
            
            if remove:
                initial_rows = len(cleaned_df)
                cleaned_df = cleaned_df.drop_duplicates()
                rows_removed = initial_rows - len(cleaned_df)
                cleaning_summary['rows_removed'] += rows_removed
                cleaning_summary['operations'].append(f"Removed {rows_removed} duplicate rows")
                print(f"   ✅ Removed {rows_removed} duplicate rows")
            else:
                print(f"   ⏭️  Skipped removal")
        else:
            print(f"   ✅ No duplicate rows found")
    else:
        print(f"\n🔄 Step 2: Duplicate removal disabled (REMOVE_DUPLICATE_ROWS = False)")
    
    # ========================================================================
    # STEP 3: Remove low variance columns
    # ========================================================================
    if REMOVE_LOW_VARIANCE_COLS:
        print(f"\n📉 Step 3: Checking for low variance columns (≥{LOW_VARIANCE_THRESHOLD:.0%} same values)...")
        
        low_var_cols = []
        for col in cleaned_df.columns:
            if len(cleaned_df[col]) > 0:
                most_common_pct = cleaned_df[col].value_counts(normalize=True).iloc[0] if len(cleaned_df[col].value_counts()) > 0 else 0
                if most_common_pct >= LOW_VARIANCE_THRESHOLD:
                    low_var_cols.append(col)
        
        if low_var_cols:
            print(f"   Found {len(low_var_cols)} low variance columns:")
            for col in low_var_cols[:10]:
                most_common_pct = cleaned_df[col].value_counts(normalize=True).iloc[0]
                print(f"      {col}: {most_common_pct:.1%} same value")
            if len(low_var_cols) > 10:
                print(f"      ... and {len(low_var_cols)-10} more")
            
            remove = True
            if interactive:
                response = input(f"\n   Remove these {len(low_var_cols)} columns? (y/n): ").lower().strip()
                remove = (response == 'y')
            
            if remove:
                cleaned_df = cleaned_df.drop(columns=low_var_cols)
                cleaning_summary['columns_removed'].extend(low_var_cols)
                cleaning_summary['operations'].append(f"Removed {len(low_var_cols)} low variance columns")
                print(f"   ✅ Removed {len(low_var_cols)} low variance columns")
            else:
                print(f"   ⏭️  Skipped removal")
        else:
            print(f"   ✅ No low variance columns found")
    else:
        print(f"\n📉 Step 3: Low variance removal disabled (REMOVE_LOW_VARIANCE_COLS = False)")
    
    # ========================================================================
    # STEP 4: Remove datetime columns
    # ========================================================================
    if REMOVE_DATE_COLUMNS:
        print(f"\n📅 Step 4: Checking for datetime columns...")
        
        datetime_cols = cleaned_df.select_dtypes(include=['datetime64', 'datetime']).columns.tolist()
        
        if datetime_cols:
            print(f"   Found {len(datetime_cols)} datetime columns: {datetime_cols}")
            
            remove = True
            if interactive:
                response = input(f"   Remove datetime columns? (y/n): ").lower().strip()
                remove = (response == 'y')
            
            if remove:
                cleaned_df = cleaned_df.drop(columns=datetime_cols)
                cleaning_summary['columns_removed'].extend(datetime_cols)
                cleaning_summary['operations'].append(f"Removed {len(datetime_cols)} datetime columns")
                print(f"   ✅ Removed {len(datetime_cols)} datetime columns")
            else:
                print(f"   ⏭️  Skipped removal")
        else:
            print(f"   ✅ No datetime columns found")
    else:
        print(f"\n📅 Step 4: Datetime removal disabled (REMOVE_DATE_COLUMNS = False)")
    
    # ========================================================================
    # STEP 5: Remove high cardinality columns (likely IDs)
    # ========================================================================
    print(f"\n🔢 Step 5: Checking for high cardinality columns (≥{HIGH_CARDINALITY_THRESHOLD:.0%} unique values)...")
    
    high_card_cols = []
    for col in cleaned_df.columns:
        if len(cleaned_df[col]) > 0:
            unique_ratio = cleaned_df[col].nunique() / len(cleaned_df[col])
            if unique_ratio >= HIGH_CARDINALITY_THRESHOLD:
                high_card_cols.append(col)
    
    if high_card_cols:
        print(f"   Found {len(high_card_cols)} high cardinality columns (likely IDs):")
        for col in high_card_cols[:10]:
            unique_ratio = cleaned_df[col].nunique() / len(cleaned_df[col])
            print(f"      {col}: {unique_ratio:.1%} unique values")
        if len(high_card_cols) > 10:
            print(f"      ... and {len(high_card_cols)-10} more")
        
        remove = True
        if interactive:
            response = input(f"\n   Remove these {len(high_card_cols)} columns? (y/n): ").lower().strip()
            remove = (response == 'y')
        
        if remove:
            cleaned_df = cleaned_df.drop(columns=high_card_cols)
            cleaning_summary['columns_removed'].extend(high_card_cols)
            cleaning_summary['operations'].append(f"Removed {len(high_card_cols)} high cardinality columns")
            print(f"   ✅ Removed {len(high_card_cols)} high cardinality columns")
        else:
            print(f"   ⏭️  Skipped removal")
    else:
        print(f"   ✅ No high cardinality columns found")
    
    # ========================================================================
    # FINAL SUMMARY
    # ========================================================================
    cleaning_summary['final_shape'] = cleaned_df.shape
    
    print(f"\n" + "=" * 60)
    print("✅ AUTOMATED DATA CLEANING COMPLETE!")
    print(f"   Original shape: {cleaning_summary['original_shape']}")
    print(f"   Final shape: {cleaning_summary['final_shape']}")
    print(f"   Columns removed: {len(cleaning_summary['columns_removed'])}")
    print(f"   Rows removed: {cleaning_summary['rows_removed']}")
    print(f"   Operations performed: {len(cleaning_summary['operations'])}")
    
    if cleaning_summary['operations']:
        print(f"\n   📋 Summary of operations:")
        for op in cleaning_summary['operations']:
            print(f"      • {op}")
    
    # Check for remaining missing values
    remaining_missing = cleaned_df.isnull().sum().sum()
    if remaining_missing > 0:
        print(f"\n   ⚠️  Remaining missing values: {remaining_missing}")
    else:
        print(f"\n   ✅ No missing values remaining")
    
    print("=" * 60)
    
    return cleaned_df, cleaning_summary

In [29]:
# Complete Data Preprocessing Workflow
# ====================================

def complete_data_preprocessing_workflow():
    """
    Complete workflow that combines all preprocessing steps in the correct order.
    Now includes the recommended strategic imputation at Decision Point 1.
    """
    
    print("🚀 COMPLETE DATA PREPROCESSING WORKFLOW (WITH STRATEGIC IMPUTATION)")
    print("=" * 70)
    
    # Step 1: Check if data is loaded
    if 'base_data' not in globals():
        print("❌ No data loaded. Please run the file selection cell first.")
        return None, None, None
    
    print(f"✅ Data loaded: {base_data.shape}")
    
    # Step 2: Run comprehensive exploration (includes string null handling)
    print("\n📊 Step 1: Running comprehensive data exploration...")
    comprehensive_data_exploration()
    
    # Step 3: Apply strategic imputation (RECOMMENDED DECISION POINT)
    print("\n🎯 Step 2: Applying strategic imputation (Decision Point 1)...")
    try:
        imputed_data, imputation_report = intelligent_imputation_strategy(
            base_data, column_info, target_var=None
        )
        print(f"✅ Strategic imputation completed!")
        print(f"   Missing values reduced: {imputation_report['missing_reduced']:,}")
        print(f"   Columns processed: {len(imputation_report['columns_imputed'])}")
        
        # Use imputed data for subsequent steps
        data_for_cleaning = imputed_data
        
    except Exception as e:
        print(f"⚠️ Imputation failed: {e}")
        print("   Proceeding with original data...")
        data_for_cleaning = base_data
        imputation_report = None
    
    # Step 4: Perform automated cleaning on imputed data
    print("\n🧹 Step 3: Performing automated data cleaning...")
    cleaned_data, cleaning_summary = automated_data_cleaning(data_for_cleaning, column_info, interactive=False)
    
    # Step 5: Target variable selection
    print(f"\n🎯 Step 4: Select target variable...")
    print("Available numeric columns for target selection:")
    numeric_cols = cleaned_data.select_dtypes(include=['float64', 'float32', 'int64', 'int32']).columns.tolist()
    for i, col in enumerate(numeric_cols[:10], 1):  # Show first 10
        print(f"  {i}. {col}")
    if len(numeric_cols) > 10:
        print(f"  ... and {len(numeric_cols)-10} more columns")
    
    print(f"\n💡 Next step: Run select_dependent_variable(cleaned_data) to choose your target")
    
    # Step 6: Final quality check
    print(f"\n🔍 Step 5: Final quality check...")
    print(f"   Original shape: {base_data.shape}")
    if imputation_report:
        print(f"   After imputation: {data_for_cleaning.shape}")
    print(f"   Final cleaned shape: {cleaned_data.shape}")
    print(f"   Columns removed: {len(cleaning_summary['columns_removed'])}")
    print(f"   Rows removed: {cleaning_summary['rows_removed']}")
    
    # Check for remaining issues
    remaining_missing = cleaned_data.isnull().sum().sum()
    if remaining_missing > 0:
        print(f"   ⚠️  Remaining missing values: {remaining_missing}")
    else:
        print(f"   ✅ No missing values remaining")
    
    # Data type summary
    dtype_summary = cleaned_data.dtypes.value_counts().to_dict()
    print(f"   📝 Final data types: {dtype_summary}")
    
    print(f"\n✅ Enhanced preprocessing complete with strategic imputation!")
    print("📋 Next steps:")
    print("   1. Run: dependent_var = select_dependent_variable(cleaned_data)")
    print("   2. Run: model_ready_data, report = advanced_string_preprocessing_for_modeling(cleaned_data, dependent_var)")
    
    # Return both cleaning and imputation results
    results = {
        'cleaned_data': cleaned_data,
        'cleaning_summary': cleaning_summary,
        'column_info': column_info,
        'imputation_report': imputation_report
    }
    
    return cleaned_data, results

# Execute complete workflow with strategic imputation
print("🔧 Ready to run enhanced preprocessing workflow with strategic imputation!")
print("\n🎯 This workflow includes:")
print("   ✅ Data loading and validation")
print("   ✅ Comprehensive analysis with string null conversion") 
print("   ✅ Strategic imputation at Decision Point 1 (RECOMMENDED)")
print("   ✅ Automated data cleaning")
print("   ✅ Quality reporting and next steps")
print("\nUncomment the line below to execute:")
print("# cleaned_data, results = complete_data_preprocessing_workflow()")

🔧 Ready to run enhanced preprocessing workflow with strategic imputation!

🎯 This workflow includes:
   ✅ Data loading and validation
   ✅ Comprehensive analysis with string null conversion
   ✅ Strategic imputation at Decision Point 1 (RECOMMENDED)
   ✅ Automated data cleaning
   ✅ Quality reporting and next steps

Uncomment the line below to execute:
# cleaned_data, results = complete_data_preprocessing_workflow()


In [44]:
cleaned_data, results = complete_data_preprocessing_workflow()

🚀 COMPLETE DATA PREPROCESSING WORKFLOW (WITH STRATEGIC IMPUTATION)
✅ Data loaded: (12383, 210)

📊 Step 1: Running comprehensive data exploration...
📊 COMPREHENSIVE DATA EXPLORATION

🔧 Preprocessing: Converting string null values to pandas null...
✅ Using 46 configured string null patterns
✅ No string null values found to convert



📊 Dataset Overview:
Shape: 12,383 rows × 210 columns
Memory usage: 58.47 MB
✅ No duplicate rows found


🔍 Data Types Analysis:
  float64: 121 columns
  object: 55 columns
  int64: 34 columns


❗ Missing Values Analysis:
Columns with missing values: 21

Top 20 columns with most missing values:
  R4_16MA[ECD_GMZ_1000_SL]@50%@ETEST: 76 (0.6%)
  PRODUCT@ETEST: 76 (0.6%)
  CHAMBER@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  SUBENTITY@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  RECIPE@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  PROCESS_TIME@CHAMBER@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  SLOT@NTSC@Process-5@CVD@T_GMZILD: 4 (0.0%)
  END_TIME@CHAMBER@NTSC@Process-5@CVD@T_GMZILD

## Advanced String Preprocessing for Modeling

After selecting your target variable, this section converts all categorical data to numeric formats suitable for machine learning algorithms.

In [45]:
def advanced_string_preprocessing_for_modeling(df, dependent_var=None):
    """
    Advanced preprocessing specifically for string columns before modeling.
    This builds on the string null conversion already implemented.
    
    Parameters:
        df: DataFrame to process
        dependent_var: Name of dependent variable (to preserve it)
    
    Returns:
        tuple: (processed_df, preprocessing_report)
    """
    
    print("🔧 ADVANCED STRING PREPROCESSING FOR MODELING")
    print("=" * 60)
    
    processed_df = df.copy()
    report = {
        'original_shape': df.shape,
        'operations': [],
        'columns_modified': [],
        'encoding_maps': {}
    }
    
    # Get string/object columns (excluding the dependent variable)
    string_cols = processed_df.select_dtypes(include=['object']).columns.tolist()
    if dependent_var and dependent_var in string_cols:
        string_cols.remove(dependent_var)
    
    print(f"📝 Found {len(string_cols)} string columns to process")
    
    # ========================================================================
    # STEP 1: Additional String Cleaning
    # ========================================================================
    print(f"\n🧹 Step 1: Additional string cleaning...")
    
    for col in string_cols:
        initial_nulls = processed_df[col].isnull().sum()
        
        # Strip whitespace and standardize case
        processed_df[col] = processed_df[col].astype(str).str.strip().str.lower()
        
        # Convert empty strings created by stripping to NaN
        processed_df[col] = processed_df[col].replace('', np.nan)
        processed_df[col] = processed_df[col].replace('nan', np.nan)  # Handle string 'nan' from astype(str)
        
        final_nulls = processed_df[col].isnull().sum()
        new_nulls = final_nulls - initial_nulls
        
        if new_nulls > 0:
            print(f"  {col}: Created {new_nulls} additional nulls from empty/whitespace strings")
    
    report['operations'].append("Cleaned whitespace and standardized case")
    
    # ========================================================================
    # STEP 2: Categorical Encoding Strategy
    # ========================================================================
    print(f"\n🏷️  Step 2: Analyzing categorical columns for encoding...")
    
    low_cardinality_cols = []
    high_cardinality_cols = []
    very_high_cardinality_cols = []
    
    for col in string_cols:
        if col in processed_df.columns:  # Check if column still exists
            unique_count = processed_df[col].nunique()
            total_count = processed_df[col].count()
            
            if total_count == 0:
                continue
                
            cardinality_ratio = unique_count / total_count
            
            if unique_count <= 10:
                low_cardinality_cols.append(col)
                print(f"  {col}: {unique_count} unique values → Good for One-Hot Encoding")
            elif unique_count <= 50:
                high_cardinality_cols.append(col)
                print(f"  {col}: {unique_count} unique values → Consider Target/Label Encoding")
            else:
                very_high_cardinality_cols.append(col)
                print(f"  {col}: {unique_count} unique values ({cardinality_ratio:.1%} unique) → Likely ID, consider dropping")
    
    # ========================================================================
    # STEP 3: Apply Encoding Strategies
    # ========================================================================
    print(f"\n🔄 Step 3: Applying encoding strategies...")
    
    # One-hot encode low cardinality columns
    if low_cardinality_cols:
        print(f"\n📊 One-hot encoding {len(low_cardinality_cols)} low-cardinality columns...")
        
        for col in low_cardinality_cols:
            if col in processed_df.columns:
                # Create dummy variables
                dummies = pd.get_dummies(processed_df[col], prefix=col, dummy_na=True)
                
                # Add to dataframe and remove original
                processed_df = pd.concat([processed_df, dummies], axis=1)
                processed_df = processed_df.drop(columns=[col])
                
                # Track the encoding
                report['encoding_maps'][col] = {
                    'method': 'one_hot',
                    'new_columns': list(dummies.columns)
                }
                report['columns_modified'].append(col)
                
                print(f"  ✅ {col} → {len(dummies.columns)} dummy columns")
    
    # Label encode high cardinality columns (preserve for potential target encoding)
    if high_cardinality_cols:
        print(f"\n🏷️  Label encoding {len(high_cardinality_cols)} medium-cardinality columns...")
        
        from sklearn.preprocessing import LabelEncoder
        
        for col in high_cardinality_cols:
            if col in processed_df.columns:
                le = LabelEncoder()
                
                # Handle missing values
                non_null_mask = processed_df[col].notna()
                
                if non_null_mask.sum() > 0:  # Only if there are non-null values
                    processed_df.loc[non_null_mask, f'{col}_encoded'] = le.fit_transform(processed_df.loc[non_null_mask, col])
                    processed_df[f'{col}_encoded'] = processed_df[f'{col}_encoded'].astype('Int64')  # Nullable integer
                    
                    # Drop original column
                    processed_df = processed_df.drop(columns=[col])
                    
                    # Track the encoding
                    report['encoding_maps'][col] = {
                        'method': 'label_encoding',
                        'encoder': le,
                        'new_column': f'{col}_encoded'
                    }
                    report['columns_modified'].append(col)
                    
                    print(f"  ✅ {col} → {col}_encoded ({len(le.classes_)} categories)")
    
    # Flag very high cardinality columns for potential removal
    if very_high_cardinality_cols:
        print(f"\n⚠️  {len(very_high_cardinality_cols)} very high-cardinality columns detected:")
        for col in very_high_cardinality_cols:
            unique_count = processed_df[col].nunique()
            print(f"  {col}: {unique_count} unique values (likely ID column)")
        
        remove_high_card = input(f"\nRemove these {len(very_high_cardinality_cols)} high-cardinality columns? (y/n): ").lower().strip()
        if remove_high_card == 'y':
            processed_df = processed_df.drop(columns=very_high_cardinality_cols)
            report['operations'].append(f"Removed {len(very_high_cardinality_cols)} high-cardinality columns")
            print(f"  ✅ Removed {len(very_high_cardinality_cols)} high-cardinality columns")
    
    # ========================================================================
    # STEP 4: Final Cleanup
    # ========================================================================
    print(f"\n🧹 Step 4: Final cleanup...")
    
    # Check for any remaining object columns
    remaining_object_cols = processed_df.select_dtypes(include=['object']).columns.tolist()
    if dependent_var and dependent_var in remaining_object_cols:
        remaining_object_cols.remove(dependent_var)
    
    if remaining_object_cols:
        print(f"⚠️  Warning: {len(remaining_object_cols)} object columns remain:")
        for col in remaining_object_cols:
            print(f"  {col}: {processed_df[col].nunique()} unique values")
        
        # Option to convert remaining to string or drop
        handle_remaining = input("Convert remaining object columns to numeric hash codes? (y/n): ").lower().strip()
        if handle_remaining == 'y':
            for col in remaining_object_cols:
                # Convert to hash codes (simple numeric conversion)
                processed_df[f'{col}_hash'] = processed_df[col].astype(str).apply(lambda x: hash(x) % (10**8))
                processed_df = processed_df.drop(columns=[col])
                print(f"  ✅ {col} → {col}_hash (numeric)")
    
    # ========================================================================
    # FINAL REPORT
    # ========================================================================
    report['final_shape'] = processed_df.shape
    report['columns_added'] = processed_df.shape[1] - df.shape[1] + len(report['columns_modified'])
    
    print(f"\n" + "=" * 60)
    print("✅ ADVANCED STRING PREPROCESSING COMPLETE!")
    print(f"Original shape: {report['original_shape']}")
    print(f"Final shape: {report['final_shape']}")
    print(f"String columns processed: {len(report['columns_modified'])}")
    print(f"Operations performed: {len(report['operations'])}")
    
    # Data type summary
    final_dtypes = processed_df.dtypes.value_counts()
    print(f"\nFinal data types:")
    for dtype, count in final_dtypes.items():
        print(f"  {dtype}: {count} columns")
    
    # Check for any remaining missing values
    total_missing = processed_df.isnull().sum().sum()
    if total_missing > 0:
        print(f"\n⚠️  Remaining missing values: {total_missing}")
        cols_with_missing = processed_df.isnull().sum()
        cols_with_missing = cols_with_missing[cols_with_missing > 0]
        for col, missing_count in cols_with_missing.head(5).items():
            pct = (missing_count / len(processed_df)) * 100
            print(f"  {col}: {missing_count} ({pct:.1f}%)")
    else:
        print("\n✅ No missing values remaining!")
    
    print("=" * 60)
    
    return processed_df, report


# Quick usage example
def prepare_data_for_modeling():
    """
    Complete pipeline: exploration → cleaning → string preprocessing
    """
    if 'cleaned_data' in globals() and 'dependent_var' in globals():
        print("🚀 Running advanced string preprocessing on cleaned data...")
        model_ready_data, preprocessing_report = advanced_string_preprocessing_for_modeling(
            cleaned_data, dependent_var
        )
        
        print(f"\n📋 Preprocessing Summary:")
        print(f"  Original columns: {preprocessing_report['original_shape'][1]}")
        print(f"  Final columns: {preprocessing_report['final_shape'][1]}")
        print(f"  String columns processed: {len(preprocessing_report['columns_modified'])}")
        
        return model_ready_data, preprocessing_report
    else:
        print("❌ Please run the data cleaning workflow first.")
        print("Variables needed: 'cleaned_data' and 'dependent_var'")
        return None, None

# Uncomment to run automatically:
model_ready_data, report = prepare_data_for_modeling()

🚀 Running advanced string preprocessing on cleaned data...
🔧 ADVANCED STRING PREPROCESSING FOR MODELING
📝 Found 49 string columns to process

🧹 Step 1: Additional string cleaning...

🏷️  Step 2: Analyzing categorical columns for encoding...
  LOT: 831 unique values (6.7% unique) → Likely ID, consider dropping
  FDC_SUBENTITY@ETCHER@255053: 35 unique values → Consider Target/Label Encoding
  FDC_ENTITY@ETCHER@255053: 12 unique values → Consider Target/Label Encoding
  PRODUCT@ETEST: 7 unique values → Good for One-Hot Encoding
  CHAMBER@NTSC@LoadLock-1@CVD@T_GMZILD: 29 unique values → Consider Target/Label Encoding
  CHAMBER@NTSC@LoadLock-2@CVD@T_GMZILD: 29 unique values → Consider Target/Label Encoding
  CHAMBER@NTSC@Process-1@CVD@T_GMZILD: 28 unique values → Consider Target/Label Encoding
  CHAMBER@NTSC@Process-2@CVD@T_GMZILD: 28 unique values → Consider Target/Label Encoding
  CHAMBER@NTSC@Process-3@CVD@T_GMZILD: 28 unique values → Consider Target/Label Encoding
  CHAMBER@NTSC@Process

## 📖 Complete Workflow Documentation

### 🎯 Summary: End-to-End Data Preprocessing Pipeline

This notebook implements a comprehensive 4-stage preprocessing pipeline:

### **Stage 1: Environment Setup**
- **Package Installation**: Intel proxy-compatible package installation
- **Library Imports**: All necessary libraries for data processing and modeling
- **Configuration**: Global constants controlling all preprocessing behaviors

### **Stage 2: Data Loading & Initial Quality**
- **File Selection**: Interactive CSV file picker with encoding handling
- **Missing Data Removal**: Automatic removal of columns exceeding missing data threshold
- **Basic Validation**: Shape, memory usage, and data type overview

### **Stage 3: Comprehensive Analysis & Cleaning**
- **String Null Conversion**: Converts 20+ string null patterns to pandas NaN
- **Data Exploration**: 10-section analysis covering all data quality aspects
- **Automated Cleaning**: Systematic removal of problematic columns based on analysis
- **Quality Reporting**: Detailed reporting of all transformations applied

### **Stage 4: Modeling Preparation**
- **Target Selection**: Interactive selection of dependent variable
- **Categorical Encoding**: Smart encoding based on cardinality
  - ≤10 unique values → One-hot encoding
  - 11-50 unique values → Label encoding  
  - >50 unique values → Flagged for removal (likely IDs)
- **Final Cleanup**: Ensures all data is numeric and model-ready

### ? **Quick Start Guide**

**Option 1: Step-by-Step Execution**
```python
# 1. Load your data
base_data = select_csv_file()

# 2. Run comprehensive analysis (includes string null handling)
comprehensive_data_exploration()

# 3. Clean the data
cleaned_data, summary = automated_data_cleaning(base_data, column_info)

# 4. Select target variable
dependent_var = select_dependent_variable(cleaned_data)

# 5. Prepare for modeling
model_ready_data, report = advanced_string_preprocessing_for_modeling(cleaned_data, dependent_var)
```

**Option 2: Automated Workflow**
```python
# Run the complete workflow automatically
cleaned_data, summary, analysis = complete_data_preprocessing_workflow()

# Then select target and finalize
dependent_var = select_dependent_variable(cleaned_data)
model_ready_data, report = advanced_string_preprocessing_for_modeling(cleaned_data, dependent_var)
```

### ⚙️ **Configuration Options**
Modify these constants to control preprocessing behavior:
- `MAX_MISSING_DATA = 0.5` → Remove columns with >50% missing data
- `HIGH_CARDINALITY_THRESHOLD = 0.8` → ID detection threshold
- `REMOVE_DUPLICATE_ROWS = True` → Automatic duplicate removal
- `REMOVE_LOW_VARIANCE_COLS = True` → Remove constant columns
- `CONVERT_STRING_NULLS = True` → Enable string null conversion

### 🎉 **Expected Results**
After completing the pipeline, you'll have:
- ✅ **Clean dataset** with no missing values or duplicates
- ✅ **Numeric data only** (suitable for ML algorithms)
- ✅ **Proper encoding** of categorical variables
- ✅ **Selected target variable** for modeling
- ✅ **Detailed reports** of all transformations applied
- ✅ **Model-ready data** for feature reduction and machine learning

The final `model_ready_data` DataFrame will be ready for feature selection, dimensionality reduction, and machine learning model training.